# Shape-Net

A simple neural network that can distinguish shapes.

## Binary Shape Net
First, we create a simple nueral net that can distinguish L-shapes from non L-shapes in 3x3 grayscale images.

### 📐 Data Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define dataset
L_shapes = [
    torch.tensor([[1,0,0],
                  [1,0,0],
                  [1,1,0]]),  # L
    torch.tensor([[1,1,1],
                  [1,0,0],
                  [0,0,0]]),  # rotated L
    torch.tensor([[0,0,1],
                  [0,0,1],
                  [0,1,1]]),  # another rotation
    torch.tensor([[0,0,0],
                  [0,0,1],
                  [1,1,1]]),  # another rotation
]

non_L_shapes = [
    torch.tensor([[1,1,0],
                  [1,1,0],
                  [0,0,0]]),  # square
    torch.tensor([[1,0,0],
                  [1,1,0],
                  [0,0,0]]),  # corner
    torch.tensor([[1,1,1],
                  [0,1,0],
                  [0,0,0]]),  # T-shape
    torch.tensor([[0,1,0],
                  [0,1,0],
                  [0,1,0]]),  # column
    torch.tensor([[1,1,1],
                  [0,0,0],
                  [0,0,0]]),  # sideways column
    torch.tensor([[0,0,1],
                  [0,0,1],
                  [0,0,1]]),  # column
    torch.tensor([[1,0,0],
                  [1,1,0],
                  [1,0,0]]),  # T
    torch.tensor([[1,0,0],
                  [0,0,0],
                  [1,1,0]]),  # Near-L
]

X = torch.stack([x.float() for x in L_shapes + non_L_shapes]).unsqueeze(1)  # Add channel dimension (1 for grayscale) -> (B, 1, H, W)
y = torch.tensor([1]*len(L_shapes) + [0]*len(non_L_shapes)).float().unsqueeze(1)

### 🧠 Model

In [ ]:
from typing import Any


class ShapeNet(nn.Module):
    def __init__(self):
        super(ShapeNet, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=2)  # 4 filters of size 2x2. One input channel (grayscale)
        self.fc = nn.Linear(4 * 2 * 2, 1)  # Flattened 4 channels × 2×2 patch → 1 output

    def forward(self, x) -> torch.Tensor:
        x = torch.relu(self.conv(x))  # (B, 4, 2, 2)
        x = x.view(x.size(0), -1)  # Flatten to (B, 16)
        x = torch.sigmoid(self.fc(x))
        return x

    # This is only done to force intelliSense to recognize the return type as torch.Tensor
    def __call__(self, *args: Any, **kwds: Any) -> torch.Tensor:
        x = super().__call__(*args, **kwds)
        if isinstance(x, torch.Tensor):
            return x
        raise TypeError(f"Expected torch.Tensor, got {type(x)}")

### ⚙️ Training

In [ ]:
model = ShapeNet()
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1000):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print("Final predictions:", y_pred.detach().round().squeeze())

### 🧪 Create Test Set

In [ ]:
# L-shaped (positive class)
test_L = [
    torch.tensor([[1,0,0],
                  [1,0,0],
                  [1,1,0]]),  # L
    torch.tensor([[0,0,0],
                  [0,0,1],
                  [1,1,1]]),  # rotated L
]

# Non-L (negative class)
test_non_L = [
    torch.tensor([[1,1,0],
                  [0,1,1],
                  [0,0,0]]),  # diagonal blob
    torch.tensor([[1,0,0],
                  [1,0,0],
                  [1,0,0]]),  # column
]

# Format test set
X_test = torch.stack([x.float() for x in test_L + test_non_L]).unsqueeze(1)  # Add channel dimension (1 for grayscale) -> (B, 1, H, W)
y_test = torch.tensor([1]*len(test_L) + [0]*len(test_non_L)).float().unsqueeze(1)

### ✅ Test Accuracy

In [ ]:
with torch.no_grad():  # Disable gradient tracking for inference
    y_pred = model(X_test)
    print("Test predictions (raw):", y_pred.squeeze())
    y_pred_labels = (y_pred >= 0.5).float()  # Threshold at 0.5
    print("Test predictions:", y_pred_labels.squeeze())
    correct = (y_pred_labels == y_test).sum().item()
    accuracy = correct / len(y_test)

print(f"Accuracy: {accuracy*100:.2f}%")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Convert to CPU lists
y_true = y_test.cpu().tolist()
y_pred = y_pred_labels.cpu().tolist()

# Compute matrix
cm = confusion_matrix(y_true, y_pred)

# Plot
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Not L", "L"], yticklabels=["Not L", "L"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()